In [1]:
import numpy as np
import gvar as gv
import sys

#import warnings  
#warnings.simplefilter(action="default")

sys.path.append('../')

from h5glance import H5Glance 
import os
import lsqfit

In [2]:
os.chdir('../../hyperon-xpt-master/xpt')

In [3]:
#import fit_analysis as fa 
import fit_routine as fit
import i_o 

In [4]:
from importlib import reload
reload(i_o)
#reload(fa)
reload(fit)

<module 'fit_routine' from '/Users/grantdb/hyperon-xpt-master/xpt/fit_routine.py'>

In [5]:
file = '../data/hyperon_data.h5'
#H5Glance(file)

In [6]:
file2 = '../data/input_data.h5'
#H5Glance(file2)

In [7]:
input_output = i_o.InputOutput()
data,ensembles = input_output.get_data(scheme='w0_imp')

#for j,ens in enumerate(ensembles):
    #print(ens, data['m_xi'][j])

/Users/grantdb/hyperon-xpt-master/xpt/i_o.py:101: UserWarning: sample sizes differ for different entries: 5000 5001
  gv_data[ens] = gv.dataset.avg_data(gv_data[ens], bstrap=True)


In [8]:
prior = {}

# not-even leading order 
prior['m_{xi,0}'] = gv.gvar(1000, 1000) # MeV
prior['m_{xi_st,0}'] = gv.gvar(1100, 1000) # MeV

# lo
prior['s_{xi}'] = gv.gvar(0, 5)
prior['s_{xi,bar}'] = gv.gvar(0, 5)

# nlo
prior['g_{xi,xi}'] = gv.gvar(0.25, 0.25)
prior['g_{xi_st,xi}'] = gv.gvar(0.25, 0.25)
prior['g_{xi_st,xi_st}'] = gv.gvar(0, 3)

# n2lo
prior['b_{xi,4}'] = gv.gvar(0, 5)
prior['b_{xi_st,4}'] = gv.gvar(0, 5)
prior['a_{xi,4}'] = gv.gvar(0, 5)
prior['a_{xi_st,4}'] = gv.gvar(0, 5)
# note: no lo terms for taylor 

# latt/strange nlo
prior['d_{xi,a}'] = gv.gvar(0, 1)
prior['d_{xi_st,a}'] = gv.gvar(0, 1) 
prior['d_{xi,s}'] = gv.gvar(0, 3)
prior['d_{xi_st,s}'] = gv.gvar(0, 3) 

# latt n2lo
prior['d_{xi,aa}'] = gv.gvar(0, 3)
prior['d_{xi,al}'] = gv.gvar(0, 6)
prior['d_{xi,as}'] = gv.gvar(0, 3)
prior['d_{xi,ls}'] = gv.gvar(0, 3)
prior['d_{xi,ss}'] = gv.gvar(0, 3)

prior['d_{xi_st,aa}'] = gv.gvar(0, 3)
prior['d_{xi_st,al}'] = gv.gvar(0, 3) 
prior['d_{xi_st,as}'] = gv.gvar(0, 3)
prior['d_{xi_st,ls}'] = gv.gvar(0, 3) 
prior['d_{xi_st,ss}'] = gv.gvar(0, 3)

In [9]:
model_info = {}
model_info['particles'] = ['xi', 'xi_st']
#model_info['name'] = 'test'
model_info['order_chiral']  = 'n2lo'
model_info['order_disc'] = 'n2lo'
model_info['order_strange'] = 'n2lo'
model_info['include_log'] = True
model_info['xpt'] = True

In [10]:
def _make_prior(data,prior):
    
    new_prior = {}
    for key in prior:
        new_prior[key] = prior[key]
    for key in ['m_pi', 'm_k', 'lam_chi', 'a/w', 'm_delta']:
        new_prior[key] = data[key]
    return new_prior

In [11]:

data,ensembles = input_output.get_data(scheme='w0_imp')
#model_info = model_info
prior = _make_prior(data,prior)
phys_point_data = input_output.get_phys_point_data

In [12]:
#run this until it works... 
fit_test = fit.fit_routine(prior=prior, data=data, model_info=model_info)

In [13]:
print(fit_test.fit)

Least Square Fit:
  chi2/dof [dof] = 0.58 [30]    Q = 0.97    logGBF = -149.97

Parameters:
       m_{xi,0}        1313 (24)       [  1000 (1000) ]  
    m_{xi_st,0}        1523 (30)       [  1100 (1000) ]  
         s_{xi}        0.75 (40)       [    0.0 (5.0) ]  
     s_{xi,bar}     -0.05 (5.00)       [    0.0 (5.0) ]  
      g_{xi,xi}        0.31 (25)       [    0.25 (25) ]  
   g_{xi_st,xi}        0.19 (25)       [    0.25 (25) ]  
g_{xi_st,xi_st}   -0.006 (2.992)       [    0.0 (3.0) ]  
       b_{xi,4}      0.0060 (80)       [    0.0 (5.0) ]  
    b_{xi_st,4}     -0.0021 (54)       [    0.0 (5.0) ]  
       a_{xi,4}        1.3 (3.5)       [    0.0 (5.0) ]  
    a_{xi_st,4}       -6.4 (3.9)       [    0.0 (5.0) ]  *
       d_{xi,a}       -0.50 (23)       [    0.0 (1.0) ]  
    d_{xi_st,a}        0.16 (28)       [    0.0 (1.0) ]  
       d_{xi,s}      0.01 (3.00)       [    0.0 (3.0) ]  
    d_{xi_st,s}      0.08 (3.00)       [    0.0 (3.0) ]  
      d_{xi,aa}        1.14 (83)     

In [14]:
fit_analysis = fa.fit_analysis(
    phys_point_data=phys_point_data, 
    data=data, 
    model_info=model_info,
    prior=prior)

NameError: name 'fa' is not defined

In [114]:
print(fit_analysis)